<a href="https://colab.research.google.com/github/Savvy-Slowley/Customer-Chatbox/blob/main/Customer_Chatbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pip Install

In [38]:
!pip install reportlab
!pip install Faker
!pip install PyPDF2
!pip install pdf2image
# This package contains the pdftoppm command, which is used by pdf2image to convert PDF files to images
!sudo apt-get install -y poppler-utils
!pip install --upgrade google-cloud-vision
!pip install spacy
!python -m spacy download en_core_web_sm

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.
2023-07-31 04:18:44.158784: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import random
from datetime import datetime, timedelta
from io import BytesIO
import os
from faker import Faker
import pandas as pd
from bs4 import BeautifulSoup
from PIL import Image
import requests
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
from reportlab.pdfgen import canvas
from PyPDF2 import PdfMerger, PdfReader
from pdf2image import convert_from_path
from google.cloud import vision
from getpass import getpass
import pickle
import re
import spacy
from fastapi import FastAPI, Request
from pyngrok import ngrok
import uvicorn
import nest_asyncio
from typing import List, Dict, Any
import json

# Creating Fake Data

In [3]:
fake = Faker()

def random_date(start, end):
    return start + timedelta(seconds=random.randint(0, int((end - start).total_seconds())))

# Specify the number of applicants and starting applicant ID
num_customers = 5
start_customer_id = 1

# Constants representing age limits
AGE_OF_ADULTHOOD = 18
AGE_OF_RETIREMENT = 65

# Constants representing state and city match percentages
STATE_MATCH_PERCENTAGE = 0.8
CITY_MATCH_PERCENTAGE = 0.2

In [4]:
# Define area codes for relevant states
area_codes = {
    'Alabama': ['205', '251', '256', '334', '938'],
    'Alaska': ['907'],
    'Arizona': ['480', '520', '602', '623', '928'],
    'Arkansas': ['479', '501', '870'],
    'California': ['209', '213', '310', '323', '408', '415', '510', '530', '559', '562', '619', '626', '650', '707', '714', '760', '805', '818', '831', '858', '909', '916', '925', '949', '951'],
    'Colorado': ['303', '719', '720', '970'],
    'Connecticut': ['203', '475', '860', '959'],
    'Delaware': ['302'],
    'Florida': ['239', '305', '321', '352', '386', '407', '561', '727', '754', '772', '786', '813', '850', '863', '904', '941', '954'],
    'Georgia': ['229', '404', '470', '478', '678', '706', '762', '770', '912'],
    'Hawaii': ['808'],
    'Idaho': ['208'],
    'Illinois': ['217', '224', '309', '312', '331', '618', '630', '708', '773', '779', '815', '847', '872'],
    'Indiana': ['219', '260', '317', '463', '574', '765', '812', '930'],
    'Iowa': ['319', '515', '563', '641', '712'],
    'Kansas': ['316', '620', '785', '913'],
    'Kentucky': ['270', '364', '502', '606', '859'],
    'Louisiana': ['225', '318', '337', '504', '985'],
    'Maine': ['207'],
    'Maryland': ['227', '240', '301', '410', '443', '667'],
    'Massachusetts': ['339', '351', '413', '508', '617', '774', '781', '857', '978'],
    'Michigan': ['231', '248', '269', '313', '517', '586', '616', '734', '810', '906', '947', '989'],
    'Minnesota': ['218', '320', '507', '612', '651', '763', '952'],
    'Mississippi': ['228', '601', '662', '769'],
    'Missouri': ['314', '417', '573', '636', '660', '816'],
    'Montana': ['406'],
    'Nebraska': ['308', '402', '531'],
    'Nevada': ['702', '725', '775'],
    'New Hampshire': ['603'],
    'New Jersey': ['201', '551', '609', '640', '732', '848', '856', '862', '908', '973'],
    'New Mexico': ['505', '575'],
    'New York': ['212', '315', '518', '585', '631', '716', '845', '914'],
    'North Carolina': ['252', '336', '704', '743', '828', '910', '919', '980', '984'],
    'North Dakota': ['701'],
    'Ohio': ['216', '220', '234', '330', '380', '419', '440', '513', '567', '614', '740', '937'],
    'Oklahoma': ['405', '539', '580', '918'],
    'Oregon': ['458', '503', '541', '971'],
    'Pennsylvania': ['215', '223', '267', '272', '412', '484', '570', '610', '717', '724', '814', '878'],
    'Rhode Island': ['401'],
    'South Carolina': ['803', '843', '854', '864'],
    'South Dakota': ['605'],
    'Tennessee': ['423', '615', '629', '731', '865', '901', '931'],
    'Texas': ['210', '214', '254', '281', '325', '346', '361', '409', '430', '432', '469', '512', '682', '713', '737', '806', '817', '830', '832', '903', '915', '936', '940', '956', '972', '979'],
    'Utah': ['385', '435', '801'],
    'Vermont': ['802'],
    'Virginia': ['276', '434', '540', '571', '703', '757', '804'],
    'Washington': ['206', '253', '360', '425', '509', '564'],
    'West Virginia': ['304', '681'],
    'Wisconsin': ['262', '274', '414', '534', '608', '715', '920'],
    'Wyoming': ['307']
}

In [5]:
# Creating the Passport Application Dataset
customer_data = {
    'customer_id': range(start_customer_id, start_customer_id + num_customers),
    'last_name': [],
    'first_name': [],
    'middle_name': [],
    'ssn': [],
    'date_of_birth': [],
    'gender': [],
    'place_of_birth': [],
    'contact_phone_number': [],
    'occupation': [],
    'employer_or_school': [],
    'height_ft': [],
    'height_in': [],
    'hair_color': [],
    'eye_color': [],
    'permanent_address': [],
}

start_date = datetime(1960, 1, 1)
end_date = datetime.now()

hair_colors = ['Black', 'Brown', 'Blonde', 'Red']
eye_colors = ['Brown', 'Blue', 'Green']

# Generate the country for the family to be born in
family_country = 'USA'

for applicant in range(num_customers):
    # Generate unique last names for parents
    father_last_name = fake.unique.last_name_male()
    mother_last_name = fake.unique.last_name_female()

    # Generate gender for the applicant
    gender = random.choice(['Male', 'Female'])

    # Generate unique first names based on gender
    if gender == 'Male':
        first_name = fake.unique.first_name_male()
        middle_name = fake.first_name_male()
    else:
        first_name = fake.unique.first_name_female()
        middle_name = fake.first_name_female()

    last_name = father_last_name

    ssn = fake.unique.ssn()  # Generate unique SSN

    date_of_birth = random_date(start_date, end_date).strftime('%Y-%m-%d')

    # Determine applicant's location/state and city
    location = random.choice(list(area_codes.keys()))
    city = fake.city()

    # Determine area code based on the location/state
    if location in area_codes:
        area_code = random.choice(area_codes[location])
    else:
        area_code = str(fake.random_int(200, 999))

    # Generate applicant's phone number with the assigned area code
    contact_phone_number = f'({area_code}) {fake.random_int(100, 999)}-{fake.random_int(1000, 9999)}'

    # Determine if place_of_birth and permanent_address should match state or city
    if random.random() < STATE_MATCH_PERCENTAGE:
        place_of_birth = city + ', ' + location  # Same state, different city
        permanent_address = fake.address().replace('\n', ', ')  # Same state, different city
    elif random.random() < CITY_MATCH_PERCENTAGE:
        place_of_birth = city + ', ' + location  # Same city and state
        permanent_address = fake.address().replace('\n', ', ') # Same city and state
    else:
        place_of_birth = fake.city() + ', ' + location  # Different city, same state
        permanent_address = fake.address().replace('\n', ', ')  # Different city, same state

    # Determine the age based on the date of birth
    age = (datetime.now() - datetime.strptime(date_of_birth, '%Y-%m-%d')).days // 365

    # Adjust height for different age groups
    if age < AGE_OF_ADULTHOOD:
        occupation = 'Student'
        employer_or_school = fake.company() + ' School'

        if age < 1:
            # For babies (less than 1 year old)
            height_ft = 0
            height_in = random.randint(16, 36)  # Height in inches for babies (16-36 inches)
        else:
            # For minors (1-17 years old)
            if age <= 5:
                # For toddlers and young children (1-5 years old)
                height_ft = 2
                height_in = random.randint(0, 11)  # Height in inches (0-11 inches)
            else:
                # For older children and teenagers (6-17 years old)
                height_ft = random.randint(4, 5)  # Height in feet for minors (4-5 feet)
                height_in = random.randint(0, 11)  # Height in inches (0-11 inches)
    elif age > AGE_OF_RETIREMENT:
        occupation = 'Retired'
        employer_or_school = None
        height_ft = random.randint(5, 6)  # Height in feet for retirees (5-6 feet)
        height_in = random.randint(0, 11)  # Height in inches (0-11 inches)
    else:
        occupation = fake.job()
        employer_or_school = fake.company()
        height_ft = random.randint(4, 6)  # Height in feet for adults (4-6 feet)
        height_in = random.randint(0, 11)  # Height in inches (0-11 inches)

    # Convert height to a formatted string
    height = f"{height_ft}'{height_in}\""

    hair_color = random.choice(hair_colors)
    eye_color = random.choice(eye_colors)

    customer_data['last_name'].append(last_name)
    customer_data['first_name'].append(first_name)
    customer_data['middle_name'].append(middle_name)
    customer_data['ssn'].append(ssn)
    customer_data['date_of_birth'].append(date_of_birth)
    customer_data['gender'].append(gender)
    customer_data['place_of_birth'].append(place_of_birth)
    customer_data['contact_phone_number'].append(contact_phone_number)
    customer_data['occupation'].append(occupation)
    customer_data['employer_or_school'].append(employer_or_school)
    customer_data['height_ft'].append(height_ft)
    customer_data['height_in'].append(height_in)
    customer_data['hair_color'].append(hair_color)
    customer_data['eye_color'].append(eye_color)
    customer_data['permanent_address'].append(permanent_address)

customer_df = pd.DataFrame(customer_data)

customer_df.head()

,customer_id,last_name,first_name,middle_name,ssn,date_of_birth,gender,place_of_birth,contact_phone_number,occupation,employer_or_school,height_ft,height_in,hair_color,eye_color,permanent_address
0,1,Neal,Michelle,Kathleen,731-10-9684,2005-01-19,Female,"Thomasmouth, Nebraska",(308) 635-1138,"Therapist, horticultural",Mann-Petty,4,11,Blonde,Brown,"315 Brooke Ferry Apt. 014, Bryantmouth, AK 82160"
1,2,Edwards,Linda,Leah,786-87-1867,2016-04-18,Female,"Jefferybury, Colorado",(720) 147-4738,Student,Santos Group School,4,4,Blonde,Green,"103 Caitlin Bridge, East John, OK 07285"
2,3,Ingram,Madeline,Melissa,697-54-4453,2022-12-27,Female,"Nicoleside, New York",(212) 959-8587,Student,Brandt Group School,0,24,Brown,Brown,"014 Marcus Inlet Apt. 870, West Kennethfort, N..."
3,4,Baker,Sydney,Anna,780-51-0172,2019-04-13,Female,"Wrightfurt, Illinois",(773) 254-5438,Student,Berry and Sons School,2,3,Brown,Blue,"942 Brooks Keys Apt. 707, North Nicholascheste..."
4,5,Hill,Sierra,Theresa,619-63-8118,2012-07-08,Female,"Port Heiditon, Idaho",(208) 316-3978,Student,"Stevens, Atkins and Whitehead School",4,4,Blonde,Blue,"06619 Waters Viaduct, West Autumn, NJ 97908"


In [6]:
# Creating the Order Dataset
order_data = {
    'customer_id': [],
    'order_id': [],
    'transaction_date': [],
    'product': [],
    'price': [],
    'quantity': []
}

# Define range for number of items per order
num_items_per_order = (1, 5)

start_date = datetime.now() - timedelta(days=3650)  # 10 years ago
end_date = datetime.now()

customer_ids = customer_df['customer_id'].tolist()

product_prices = {
    'Smartphone': (200, 1200),
    'Headphones': (20, 400),
    'Laptop': (300, 2000),
    'Mouse': (10, 100),
    'Smartwatch': (150, 700),
    'Fitness Tracker': (50, 300),
    'Camera': (100, 2500),
    'SD Card': (10, 200)
}

order_id = 1

while order_id <= 25:  # Generate 25 orders
    random_customer_id = random.choice(customer_ids)
    transaction_date = random_date(start_date, end_date).strftime('%Y-%m-%d')
    num_items = random.randint(*num_items_per_order)  # Determine number of items in the order

    for _ in range(num_items):  # For each item
        product = random.choice(list(product_prices.keys()))
        price_range = product_prices[product]
        price = random.randint(price_range[0], price_range[1])
        quantity = random.randint(1, 5)

        order_data['customer_id'].append(random_customer_id)
        order_data['order_id'].append(order_id)
        order_data['transaction_date'].append(transaction_date)
        order_data['product'].append(product)
        order_data['price'].append(price)
        order_data['quantity'].append(quantity)

    order_id += 1  # Only increment order_id after all items of the order are added

order_df = pd.DataFrame(order_data)
order_df.head()

,customer_id,order_id,transaction_date,product,price,quantity
0,1,1,2017-07-05,Smartphone,817,4
1,1,1,2017-07-05,Laptop,1696,4
2,1,1,2017-07-05,Headphones,20,3
3,1,1,2017-07-05,Laptop,427,5
4,1,1,2017-07-05,SD Card,195,5


# Scrape The Retailer Image From Wikipedia

In [7]:
def save_logo_from_wikipedia(url, file_name):
    try:
        # Send a GET request to fetch the page content
        response = requests.get(url)

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")

        # Find the image element in the HTML using the provided class name "infobox-image"
        image_element = soup.find("td", class_="infobox-image")

        # Extract the image URL from the img tag
        image_url = "https:" + image_element.find("img")["src"]

        # Send a GET request to fetch the image content
        image_response = requests.get(image_url)

        # Check if the image response is successful
        image_response.raise_for_status()

        # Open the image using Pillow
        image = Image.open(BytesIO(image_response.content))

        # Save the image locally
        image.save(file_name)
        print(f"Image saved: {file_name}")

    except Exception as e:
        print(f"Error processing {url}: {e}")

# list of URLs and corresponding filenames
url_file_list = [
    ("https://en.wikipedia.org/wiki/Best_Buy", 'Best_Buy_logo.png'),
    ("https://en.wikipedia.org/wiki/Walmart", 'Walmart_logo.png'),
    ("https://en.wikipedia.org/wiki/Amazon_(company)", 'Amazon_logo.png'),
]

for url, file_name in url_file_list:
    save_logo_from_wikipedia(url, file_name)


Image saved: Best_Buy_logo.png
Image saved: Walmart_logo.png
Image saved: Amazon_logo.png


# Creating Invoice PDF

In [8]:
def generate_invoice(order, customer, invoice_num):
    c = canvas.Canvas(f"invoice_{invoice_num}.pdf", pagesize=letter)
    width, height = letter

    logo_files = [file_name for logo, file_name in url_file_list]
    random_logo_file = random.choice(logo_files)
    logo = ImageReader(random_logo_file)
    c.drawImage(logo, 50, height - 160, width=100, height=100, mask='auto') # mask='auto' preserves trransparency


    c.setFont("Helvetica-Bold", 18)
    c.drawString(50, height - 210, f"Invoice for Order # {order.iloc[0]['order_id']}")

    c.setFont("Helvetica", 12)
    c.drawString(50, height - 260, f"Customer ID: {customer['customer_id']}")
    c.drawString(50, height - 280, f"Customer Name: {customer['first_name']} {customer['last_name']}")
    c.drawString(50, height - 300, f"Address: {customer['permanent_address']}")
    c.drawString(50, height - 320, f"Phone: {customer['contact_phone_number']}")
    c.drawString(50, height - 340, f"Date: {order.iloc[0]['transaction_date']}")

    # Now list the order items
    c.setFont("Helvetica-Bold", 14)
    c.drawString(50, height - 380, "Order Items:")
    c.setFont("Helvetica", 12)

    y = height - 400
    total_price = 0
    for _, row in order.iterrows():
        c.drawString(50, y, f"Product: {row['product']}, Price: ${row['price']}, Quantity: {row['quantity']}")
        total_price += row['price'] * row['quantity']
        y -= 30  # Move down the page for the next line, giving more space with each line

    # Finally, add a total
    c.setFont("Helvetica-Bold", 14)
    c.drawString(50, y - 30, f"Total: ${total_price}")

    c.save()

# Get all unique order ids
unique_order_ids = order_df['order_id'].unique()

# Generate an invoice for each order
pdfs = []
for i, order_id in enumerate(unique_order_ids):
    order = order_df[order_df['order_id'] == order_id]
    customer = customer_df[customer_df['customer_id'] == order.iloc[0]['customer_id']].iloc[0]
    generate_invoice(order, customer, i+1)
    pdfs.append(f"invoice_{i+1}.pdf")

# Merge all pdfs into one
merger = PdfMerger()

for pdf in pdfs:
    merger.append(pdf)

merger.write("all_invoices.pdf")
merger.close()

# Remove individual invoice files
for pdf in pdfs:
    os.remove(pdf)

#Input my Google Credential

In [9]:
# This line will prompt the user to input the path, which will not be displayed
service_account_path = getpass('Enter path to your service account file: ')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_path

Enter path to your service account file: ··········


#Identify the Logo on Each PDF Page

In [21]:
# Function to convert PDF to a list of images
def convert_pdf_to_images(pdf_path):
    # Convert the PDF to a list of images
    pages = convert_from_path(pdf_path)

    images = []
    for i, page in enumerate(pages):
        # Save each page as a separate image file
        image_path = f"page_{i}.png"
        page.save(image_path, 'PNG')
        images.append(image_path)

    return images

# Function to detect logos using Google Cloud Vision API
def detect_logos(image_path, page_number):
    # Initialize the Vision API client
    client = vision.ImageAnnotatorClient()

    # Read the image file
    with open(image_path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # Detect logos in the image
    response = client.logo_detection(image=image)

    # Store the logos that were detected for the corresponding page number
    logos = response.logo_annotations
    logo_descriptions_dict[page_number] = [logo.description for logo in logos]

# Open the PDF file
with open('all_invoices.pdf', 'rb') as file:
    # Create a PDF reader object
    reader = PdfReader(file)

    # Get the number of pages in the PDF
    num_pages = len(reader.pages)

    # Initialize a list to store DataFrames for each page
    page_dataframes = []

    # For each page, extract the text and create a DataFrame
    for page_number in range(num_pages):
        page = reader.pages[page_number]
        text = page.extract_text()

        # Create a DataFrame for the current page's text
        data = {'Page_Number': [page_number + 1], 'Text': [text]}
        df = pd.DataFrame(data)

        # Append the DataFrame to the list
        page_dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
pdfs_df = pd.concat(page_dataframes, ignore_index=True)

# Convert the PDF to images
pdf_path = "all_invoices.pdf"
image_paths = convert_pdf_to_images(pdf_path)

# Initialize an empty dictionary to store the logo descriptions for each page
logo_descriptions_dict = {}

# Loop through each image and detect logos, storing the results in the logo_descriptions_dict
for page_number, image_path in enumerate(image_paths):
    # Adjust page_number to match the DataFrame index (starts from 0)
    page_number_adjusted = page_number + 1
    detect_logos(image_path, page_number_adjusted)

# Add a new column "Organization" to pdfs_df with the logo descriptions
pdfs_df["Organization"] = pdfs_df["Page_Number"].map(logo_descriptions_dict)

# Clean up - Delete the images
for image_path in image_paths:
    os.remove(image_path)

Page 1:
Invoice for Order # 1
Customer ID: 1
Customer Name: Michelle Neal
Address: 315 Brooke Ferry Apt. 014, Bryantmouth, AK 82160
Phone: (308) 635-1138
Date: 2017-07-05
Order Items:
Product: Smartphone, Price: $817, Quantity: 4
Product: Laptop, Price: $1696, Quantity: 4
Product: Headphones, Price: $20, Quantity: 3
Product: Laptop, Price: $427, Quantity: 5
Product: SD Card, Price: $195, Quantity: 5
Total: $13222

Organization: ['Amazon']


In [24]:
# Print the pdfs_df DataFrame
pdfs_df.head()

,Page_Number,Text,Organization
0,1,Invoice for Order # 1\nCustomer ID: 1\nCustome...,[Amazon]
1,2,Invoice for Order # 2\nCustomer ID: 1\nCustome...,[Walmart]
2,3,Invoice for Order # 3\nCustomer ID: 5\nCustome...,[Walmart]
3,4,Invoice for Order # 4\nCustomer ID: 1\nCustome...,[Best Buy]
4,5,Invoice for Order # 5\nCustomer ID: 3\nCustome...,[Amazon]


In [25]:
# Save the pdfs_df DataFrame as a pickle file so I don't have to rerun script and nuse up Google resources
pdfs_df.to_pickle("pdfs_df.pkl")
# Read the pickle back into the script
pdfs_df = pd.read_pickle("pdfs_df.pkl")

#Convert Dataframe Into Seperate Columns

In [30]:
# Load the English language model for spaCy
nlp = spacy.load("en_core_web_sm")

# Regular expressions to identify specific patterns
invoice_pattern = re.compile(r"Invoice for Order # (\d+)")
customer_id_pattern = re.compile(r"Customer ID: (\d+)")

# Function to extract order items
def extract_order_items(text):
    order_items = []
    lines = text.split("\n")
    start_index = lines.index("Order Items:")
    for line in lines[start_index + 1:]:
        if line.startswith("Total:"):
            break
        item = re.search(r"Product: (.+), Price: \$([\d.]+), Quantity: (\d+)", line)
        if item:
            product, price, quantity = item.groups()
            order_items.append({"Product": product, "Price": float(price), "Quantity": int(quantity)})
    return order_items

# Create an empty list to hold the extracted data for each entry in pdfs_df
extracted_data = []

# Iterate through each entry in pdfs_df
for index, row in pdfs_df.iterrows():
    page_number = row['Page_Number']
    text = row['Text']
    organization = row['Organization'][0]  # Extract the actual organization name from the list

    # Extracting the Invoice for Order # using regex
    invoice_match = invoice_pattern.search(text)
    if invoice_match:
        invoice_order_number = invoice_match.group(1)
    else:
        invoice_order_number = None

    # Extracting the Customer ID using regex
    customer_id_match = customer_id_pattern.search(text)
    if customer_id_match:
        customer_id = customer_id_match.group(1)
    else:
        customer_id = None

    # Extract other information from the text
    customer_name = re.search(r"Customer Name: (.+)", text).group(1)
    address = re.search(r"Address: (.+)", text).group(1)
    phone = re.search(r"Phone: (.+)", text).group(1)
    date = re.search(r"Date: (.+)", text).group(1)
    total = re.search(r"Total: (.+)", text).group(1)
    order_items = extract_order_items(text)

    # Append the extracted data to the list
    extracted_data.append({
        "Page": page_number,
        "Organization": organization,
        "Invoice for Order #": invoice_order_number,
        "Customer ID": customer_id,
        "Customer Name": customer_name,
        "Address": address,
        "Phone": phone,
        "Date": date,
        "Order Items": order_items,
        "Total": float(total.replace("$", ""))  # Convert Total to a numeric value
    })

# Create the final DataFrame from the list of extracted data
final_df = pd.DataFrame(extracted_data)
final_df

,Page,Organization,Invoice for Order #,Customer ID,Customer Name,Address,Phone,Date,Order Items,Total
0,1,Amazon,1,1,Michelle Neal,"315 Brooke Ferry Apt. 014, Bryantmouth, AK 82160",(308) 635-1138,2017-07-05,"[{'Product': 'Smartphone', 'Price': 817.0, 'Qu...",13222.0
1,2,Walmart,2,1,Michelle Neal,"315 Brooke Ferry Apt. 014, Bryantmouth, AK 82160",(308) 635-1138,2016-08-11,"[{'Product': 'Laptop', 'Price': 674.0, 'Quanti...",7390.0
2,3,Walmart,3,5,Sierra Hill,"06619 Waters Viaduct, West Autumn, NJ 97908",(208) 316-3978,2022-08-19,"[{'Product': 'Fitness Tracker', 'Price': 175.0...",2877.0
3,4,Best Buy,4,1,Michelle Neal,"315 Brooke Ferry Apt. 014, Bryantmouth, AK 82160",(308) 635-1138,2019-11-07,"[{'Product': 'Fitness Tracker', 'Price': 283.0...",1132.0
4,5,Amazon,5,3,Madeline Ingram,"014 Marcus Inlet Apt. 870, West Kennethfort, N...",(212) 959-8587,2018-06-17,"[{'Product': 'Headphones', 'Price': 108.0, 'Qu...",4288.0
5,6,Best Buy,6,1,Michelle Neal,"315 Brooke Ferry Apt. 014, Bryantmouth, AK 82160",(308) 635-1138,2014-11-02,"[{'Product': 'Fitness Tracker', 'Price': 260.0...",6100.0
6,7,Amazon,7,3,Madeline Ingram,"014 Marcus Inlet Apt. 870, West Kennethfort, N...",(212) 959-8587,2015-09-09,"[{'Product': 'Laptop', 'Price': 1992.0, 'Quant...",22924.0
7,8,Walmart,8,3,Madeline Ingram,"014 Marcus Inlet Apt. 870, West Kennethfort, N...",(212) 959-8587,2017-11-09,"[{'Product': 'Fitness Tracker', 'Price': 179.0...",716.0
8,9,Best Buy,9,3,Madeline Ingram,"014 Marcus Inlet Apt. 870, West Kennethfort, N...",(212) 959-8587,2015-01-03,"[{'Product': 'Laptop', 'Price': 340.0, 'Quanti...",4657.0
9,10,Best Buy,10,4,Sydney Baker,"942 Brooks Keys Apt. 707, North Nicholascheste...",(773) 254-5438,2014-10-14,"[{'Product': 'Camera', 'Price': 1591.0, 'Quant...",10506.0


#Connect to Dialogflow

In [37]:
# Initialize FastAPI app
app = FastAPI()

# Define a simple route for the root of your application
@app.get("/")
def read_root():
    # This endpoint will return a simple hello world message when accessed
    return {"Hello": "Hello World"}

# Define an asynchronous post endpoint to handle Dialogflow webhook requests
@app.post("/dialogflow-webhook")
async def handle_dialogflow_webhook(request: Request):
    # Parse the JSON body of the request
    data = await request.json()

    # Extract the intent name from the parsed data
    intent_name = data['queryResult']['intent']['displayName']

    # Check if the intent name matches 'invoice_info'
    if intent_name == 'invoice_info':
        # If so, extract the parameters from the data
        parameters = data['queryResult']['parameters']

        # Try to extract the 'order_no' from the parameters, if it doesn't exist return an empty string
        order_no = parameters.get('order_no', '')

        # Handle the 'invoice_info' intent and return the response
        response = handle_invoice_info_intent(order_no)
        return response

# Define a function to handle the 'invoice_info' intent
def handle_invoice_info_intent(order_no: str):
    try:
        # Try to find a record that matches the provided 'order_no'
        result = final_df.loc[final_df['Invoice for Order #'] == int(order_no)]

        # Check if the result dataframe is empty (no matching order)
        if result.empty:
            # If so, return a message saying no data was found
            return {
                "fulfillmentText": f"No data found for Order #{order_no}",
            }

        # If a matching record was found, extract the first row
        row = result.iloc[0]

        # Return a response with the order details
        return {
            "fulfillmentText": f"Here is the information for Order #{order_no}:\n"
                               f"Organization: {row['Organization']}\n"
                               f"Customer ID: {row['Customer ID']}\n"
                               f"Customer Name: {row['Customer Name']}\n"
                               f"Address: {row['Address']}\n"
                               f"Phone: {row['Phone']}\n"
                               f"Date: {row['Date']}\n"
                               f"Order Items: {json.loads(row['Order Items'])}\n"
                               f"Total: {row['Total']}"
        }
    except Exception as e:
        # If there was an error, return a message with the error details
        return {
            "fulfillmentText": f"Sorry, there was an error processing the request. Details: {str(e)}"
        }

# Apply nest_asyncio to enable running event loops in notebook
nest_asyncio.apply()

# Establish a public URL for the application using ngrok
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

# Run the application
uvicorn.run(app, host='0.0.0.0', port=8000)

Public URL: https://ca5b-34-83-128-20.ngrok.io


INFO:     Started server process [185113]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2600:387:5:803::bb:0 - "GET / HTTP/1.1" 200 OK
INFO:     2600:387:5:803::bb:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     66.249.84.225:0 - "POST / HTTP/1.1" 405 Method Not Allowed


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [185113]
